# Processing Speech Samples with Parselmouth - mPower dataset

Aaron S. Kemp, MBA, PhD Candidate<br>
Department of Biomedical Informatics,<br> 
University of Arkansas for Medical Sciences<br>

**Licensing Note:**  
The following blocks of code were adapted from the PraatScripts GitHub repository created by Dr. David Feinberg and made available under an MIT License.  
Repository: [PraatScripts on GitHub](https://github.com/drfeinberg/PraatScripts)  
Open Science Foundation project: [OSF Link](https://osf.io/6dwr3/)

**Parselmouth Praat Scripts in Python**  
- **Contributors:** David R. Feinberg  
- **Original Creation Date:** 2018-11-17  
- **Last Updated:** 2022-01-01  
- **DOI:** [10.17605/OSF.IO/6DWR3](https://doi.org/10.17605/OSF.IO/6DWR3)  
- **Category:** Methods and Measures  
- **Description:** This script measures pitch, standard deviation of pitch, harmonics-to-noise ratio (HNR), jitter, shimmer, and formants in Python using Parselmouth, a package that runs Praat in Python.  
- **Original License: MIT License**<br>
You may obtain a copy of the License at [MIT License](https://github.com/drfeinberg/PraatScripts/blob/master/LICENSE)

**Current Adaptation by Aaron S. Kemp**  
This adaptation, based on the original code by Dr. Feinberg, has been modified for the purposes of this project and is provided under the following terms:

- **Copyright (C) 2024 University of Arkansas for Medical Sciences**  
- **Author:** Aaron S. Kemp, askemp@uams.edu  
- **Licensed under the Apache License, Version 2.0**  
You may obtain a copy of the License at [Apache License 2.0](http://www.apache.org/licenses/LICENSE-2.0)

**Additional Notes:**  
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. Please refer to both the MIT License (for the original work) and the Apache License 2.0 (for the adaptations) for specific terms and conditions regarding the use, distribution, and modification of the code. You may not use this file except in compliance with the License.

In [1]:
import parselmouth
from parselmouth.praat import call
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
from numpy import mean
from numpy import std
import statistics


In [2]:
females = pd.read_csv("females.csv")

### This function measures duration, pitch, HNR, jitter, and shimmer

In [15]:
# This is the function to measure source acoustics.

def measurePitch(sampleID, f0min, f0max, unit):
    sound = parselmouth.Sound(sampleID) # read the sound
    duration = call(sound, "Get total duration") # duration
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0, 0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    
    return duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

In [30]:
# This function measures formants using Formant Position formula

def measureFormants(sound, wave_file, f0min, f0max):
    sound = parselmouth.Sound(sound) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 4, 6000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)
    f3_mean = statistics.mean(f3_list)
    f4_mean = statistics.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_stdev = statistics.stdev(f1_list)
    f2_stdev = statistics.stdev(f2_list)
    f3_stdev = statistics.stdev(f3_list)
    f4_stdev = statistics.stdev(f4_list)
    
    return f1_mean, f2_mean, f3_mean, f4_mean, f1_stdev, f2_stdev, f3_stdev, f4_stdev

In [31]:
# create lists to put the results
file_list = []
duration_list = []
mean_F0_list = []
sd_F0_list = []
hnr_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []
f1_mean_list = []
f2_mean_list = []
f3_mean_list = []
f4_mean_list = []
f1_stdev_list = []
f2_stdev_list = []
f3_stdev_list = []
f4_stdev_list = []

# Go through all the wave files in the folder and measure all the acoustics
for wave_file in glob.glob("*.wav"):
    print(wave_file)
    if wave_file in females.values:                
        sound = parselmouth.Sound(wave_file)
        (duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(sound, 100, 600, "Hertz") #setting of pitch floor and ceiling as 100-600 for females
        (f1_mean, f2_mean, f3_mean, f4_mean, f1_stdev, f2_stdev, f3_stdev, f4_stdev) = measureFormants(sound, wave_file, 100, 600) #setting of pitch floor and ceiling as 100-600 for females
        file_list.append(wave_file) # make an ID list
        duration_list.append(duration) # make duration list
        mean_F0_list.append(meanF0) # make a mean F0 list
        sd_F0_list.append(stdevF0) # make a sd F0 list
        hnr_list.append(hnr) #add HNR data
    
        # add raw jitter and shimmer measures
        localJitter_list.append(localJitter)
        localabsoluteJitter_list.append(localabsoluteJitter)
        rapJitter_list.append(rapJitter)
        ppq5Jitter_list.append(ppq5Jitter)
        ddpJitter_list.append(ddpJitter)
        localShimmer_list.append(localShimmer)
        localdbShimmer_list.append(localdbShimmer)
        apq3Shimmer_list.append(apq3Shimmer)
        aqpq5Shimmer_list.append(aqpq5Shimmer)
        apq11Shimmer_list.append(apq11Shimmer)
        ddaShimmer_list.append(ddaShimmer)
        
        #add the formant data
        f1_mean_list.append(f1_mean)
        f2_mean_list.append(f2_mean)
        f3_mean_list.append(f3_mean)
        f4_mean_list.append(f4_mean)
        f1_stdev_list.append(f1_stdev)
        f2_stdev_list.append(f2_stdev)
        f3_stdev_list.append(f3_stdev)
        f4_stdev_list.append(f4_stdev)
        
    else:
        sound = parselmouth.Sound(wave_file)
        (duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(sound, 75, 300, "Hertz") #setting of pitch floor and ceiling as 75-300 for males
        (f1_mean, f2_mean, f3_mean, f4_mean, f1_stdev, f2_stdev, f3_stdev, f4_stdev) = measureFormants(sound, wave_file, 75, 300) #setting of pitch floor and ceiling as 75-300 for males
        file_list.append(wave_file) # make an ID list
        duration_list.append(duration) # make duration list
        mean_F0_list.append(meanF0) # make a mean F0 list
        sd_F0_list.append(stdevF0) # make a sd F0 list
        hnr_list.append(hnr) #add HNR data
    
        # add raw jitter and shimmer measures
        localJitter_list.append(localJitter)
        localabsoluteJitter_list.append(localabsoluteJitter)
        rapJitter_list.append(rapJitter)
        ppq5Jitter_list.append(ppq5Jitter)
        ddpJitter_list.append(ddpJitter)
        localShimmer_list.append(localShimmer)
        localdbShimmer_list.append(localdbShimmer)
        apq3Shimmer_list.append(apq3Shimmer)
        aqpq5Shimmer_list.append(aqpq5Shimmer)
        apq11Shimmer_list.append(apq11Shimmer)
        ddaShimmer_list.append(ddaShimmer)
        
        # add the formant data
        f1_mean_list.append(f1_mean)
        f2_mean_list.append(f2_mean)
        f3_mean_list.append(f3_mean)
        f4_mean_list.append(f4_mean)
        f1_stdev_list.append(f1_stdev)
        f2_stdev_list.append(f2_stdev)
        f3_stdev_list.append(f3_stdev)
        f4_stdev_list.append(f4_stdev)

000c901b-049b-41e1-b238-eae9c209edaa.wav
009c2bc2-87f8-4abc-8a69-66ca9ea9ebc9.wav
015681d1-b119-43a4-9e91-99321b55d9c1.wav
028a705d-8c69-4f18-9689-bb23de386f17.wav
04286562-850f-4adc-95e8-c43676dcb175.wav
05ff3854-e419-4b32-9fa9-dd00bd71c0b5.wav
061bac17-5124-42cd-bfe3-07b2ab084a2f.wav
06aadadb-b970-4f58-9864-320767e799f5.wav
072b13b9-155f-43c8-a45b-bb2a0f121050.wav
07c756b4-d0c9-4e99-9c48-4e7e2f4fab05.wav
07d6ff46-13de-4a5c-a774-c0b0eb2a9c13.wav
081ff0b8-a262-4b13-bc28-102fd0bc5b35.wav
08531203-17d3-4024-871a-d89d21ce4c6d.wav
08e36faa-ce8d-41d2-9b79-3cea44923e2c.wav
09f22556-8f3c-4c53-8db8-436a426f2668.wav
0b503cf9-1f3d-44d7-8aea-1d5e627c5c7a.wav
0bbd6d1f-d906-4a53-a745-d024f0c24848.wav
0c3471f8-a601-4488-8b35-4f73bf8cb329.wav
0d5fd350-4224-4ab2-9bfd-102db16fb2ec.wav
0e7844e2-39fc-4e8a-9108-f175f36762af.wav
0eb4a06b-5b85-4585-b50f-0a98c798529f.wav
0f0138aa-c7be-4378-bb0d-b4c3cb766f4e.wav
0fddbce1-525b-433f-84b5-c3ffe006a69a.wav
10cf5390-4f89-4dbf-8971-35ba531fb993.wav
11290522-d36a-4d

8109f152-aad2-4402-aefe-68cb6e0511c4.wav
81236ecd-bf3e-42cf-813e-56e57e5cfc1a.wav
8148bcec-9a08-4eb8-997b-85f4e2b33c20.wav
819551a2-d959-46a8-a4e2-038652627082.wav
81b5efe6-a89c-48d4-9c15-b900436fb741.wav
829df0c3-83e9-4b77-ae52-3e0af5dce4c9.wav
82f8e00a-d095-45d4-bb65-b496b5b0743c.wav
833638e8-4678-435f-aae4-1c767bda5417.wav
834f9f0c-6d24-4387-b85f-ed0d12e47a3e.wav
836bd9d2-e73a-4601-8cff-30c486c29d35.wav
8497bc51-eb0d-417d-b9c1-c735bf178db1.wav
84c4983b-1325-496d-8997-480eed062fd8.wav
854bc3d2-7e9b-451f-b0e8-67995c193aa7.wav
85bbe0d4-e6b0-4357-aabe-70beda10e151.wav
8611b500-78a0-4499-9690-f653ba36f96a.wav
863c6dd8-ac72-458d-87b9-bdcb3e787557.wav
865d2af8-8024-40f6-b477-1973bc1f293e.wav
86baa0b6-9bc3-462f-9440-fbff37cfef48.wav
87c6bc20-79f9-4d81-b06e-8975999b4ceb.wav
8859da3f-2644-4fcf-ae12-129754c33190.wav
887c7c0d-2e76-40e1-8a89-a48cb886722a.wav
88d00c1d-a3b9-42a3-a7ce-5e94029f9cc6.wav
890caf3f-49d6-40b9-be5a-4332bedea2e1.wav
894b9446-1ebc-4893-8916-5a4bbff2b854.wav
8a813c18-e635-4d

In [32]:
# Add the data to Pandas df
df = pd.DataFrame(np.column_stack([file_list, duration_list, mean_F0_list, sd_F0_list, hnr_list, 
                                   localJitter_list, localabsoluteJitter_list, rapJitter_list, 
                                   ppq5Jitter_list, ddpJitter_list, localShimmer_list, 
                                   localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, 
                                   apq11Shimmer_list, ddaShimmer_list, f1_mean_list, 
                                   f2_mean_list, f3_mean_list, f4_mean_list, 
                                   f1_stdev_list, f2_stdev_list, f3_stdev_list, 
                                   f4_stdev_list]),
                                   columns=['voiceID', 'duration', 'meanF0Hz', 'stdevF0Hz', 'HNR', 
                                            'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                            'ppq5Jitter', 'ddpJitter', 'localShimmer', 
                                            'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                            'apq11Shimmer', 'ddaShimmer', 'f1_mean', 'f2_mean', 
                                            'f3_mean', 'f4_mean', 'f1_stdev', 
                                            'f2_stdev', 'f3_stdev', 'f4_stdev'])

# Write out the updated dataframe
df.to_csv("mPower_AH_sexed_parselmouth_results_12-13-23.csv", index=False)